In [38]:
import json
import requests
import csv 

api_key_drew = 'sBfMQZWan1rpr3ugchVmpKLV_mIw9-GUMnNF52cznlGZc7GINf2J3TBP0We9j8fyIPFMsZP_if7E6H0k72AvPsMU8wwL-hOXWbW2zuRkSEa4TmBEb171AGvtbOecXXYx'
api_key_erik = 'L9JXqxrohAsyw83Y87m4Aw1IpMjOOkhw660ZaZXxpYi5cwgJD2p_EG7nbfzmE827BmPNkpZ-R3ch6J91e1thjTILnGyAV5fWygEDCx1imp-eqVMlQBHZzGSUfx-uXXYx'


In [39]:
def parse_yelp_business_data(businesses):
    params = ['id','alias','name','is_closed','review_count','rating','price',
    #          'transactions',
              'categories',         
              'coordinates',
              'location']

    header = ['id','alias','name','is_closed','review_count','rating','price',
    #          'transactions',
              'categories',
              'latitude','longitude',
              'address','city','zipcode','state','country']

    csvData = [header]
    for business in businesses: 
        data = []
        for param in params: 
            try:
                if (param == 'coordinates'):
                    coordinates = business[param]
                    data.append(coordinates['latitude'])
                    data.append(coordinates['longitude'])
                elif (param == 'location'):
                    location = business[param]
                    address = location['address1']
                    city = location['city']
                    zipcode = location['zip_code']
                    state = location['state']
                    country = location['country']
                    data.extend([address, city, zipcode, state, country])
                elif (param == 'categories'):
                    categories = business[param]
                    cats = []
                    for category in categories:
                        cats.append(category['alias'])
                    data.append(','.join([str(x) for x in cats]))
                else:    
                    value = business[param]
                    data.append(value)
            except:
                data.append('MISSING')
        if ( len(data) != 0):
            csvData.append(data)
    return csvData

In [40]:
headers = {'Authorization': 'Bearer %s' % api_key_erik}

offset = 0
limit = 10
max_limit = 50

url = 'https://api.yelp.com/v3/businesses/search'
params = {'term': 'Restaurants',
          'location': 'Brooklyn, NY',
          'limit': limit,
          'offset': offset
         }

req = requests.get(url, params=params, headers=headers)
parsed = json.loads(req.text)

businesses = parsed['businesses']
total = parsed['total']

csvData = parse_yelp_business_data(businesses)

with open('restaurant_data.csv', 'w', newline='') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerows(csvData)

csvFile.close()